In [75]:
# import libraries
import requests
import time
from datetime import datetime, date, timedelta
import json
import sys
import random
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)  #allows display of all columns in dfs

# matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import table
%matplotlib inline

# seaborn
import seaborn as sns
colors = ["#118AB2", "#EF476F", "#FFD166", "#06D6A0", "#EE754D", "#002E99"]
#plt.style.use('seaborn-whitegrid')

# plotly  https://plotly.com/python/figure-factory-table/
import plotly.figure_factory as ff

# sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [2]:
# mysql login info
sys.path.insert(0, '../../Key')
from mysql_secret import dbuser, dbpass, dbhost, dbname
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')

In [3]:
def flatten_cols(df):
    df.columns = [
        '_'.join(tuple(map(str, t))).strip() 
        for t in df.columns.values
        ]
    return df

In [69]:
def remove_comp_suffix(df, company='company'):
    """Removes traling company name info that takes up space."""
    remove_text = [' /FL', '/DE', ', Inc.', ' Inc.', ', INC.', ' INC.',' PLC', ', Corp.', 'corp.', ' Corp.', ' CORP.', 
                   ' CORP',', Ltd', ' Ltd', ' plc',  ' PLC', ' SE']
    for t in remove_text:
        df[company] = df[company].str.rstrip(t)
    
    return df    

In [90]:
# Establish Twitter connection
import tweepy

# get tweepy credentials
sys.path.insert(0, '../../Key')
from tweepy_keys import CONSUMER_KEY, CONSUMER_SECRET, ACCESS_KEY, ACCESS_SECRET

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tweepy.API(auth)

# Upcoming Week Schedule

In [63]:
# get range for today until one week from today
today = date.today()
week_from_today = today + timedelta(days=7)

#convert to string for query purposes from SQL
today = today.strftime("%Y-%m-%d")
week_from_today = week_from_today.strftime("%Y-%m-%d")

print(f"{today} to {week_from_today}")

2021-02-07 to 2021-02-14


In [64]:
connection = engine.connect()
tw_ipos_df = pd.read_sql(f"""
    SELECT *
    FROM stocks
    WHERE priced_date BETWEEN '{today}' and '{week_from_today}'
    """, connection)

In [71]:
# convert to datetime
tw_ipos_df['priced_date'] =  pd.to_datetime(tw_ipos_df['priced_date'])

# add column for month day
tw_ipos_df["price_month_day"] = tw_ipos_df['priced_date'].dt.strftime('%b %d')

# add market cap
tw_ipos_df["market_cap"] = (tw_ipos_df['dollar_val_shares'].astype(float)/1000000).round(1).astype(str) + 'M'

# filter out SPACS
tw_ipos_df = tw_ipos_df.loc[(tw_ipos_df["exchange"] != 'NASDAQ Capital')]
tw_ipos_df = tw_ipos_df.loc[~(tw_ipos_df["company"].str.contains("Acquisition", regex=False))]  #any company that contains "Acquisition"

tw_ipos_df = tw_ipos_df.sort_values("priced_date")

# remove inc. and corp. and stuff like that from company name
tw_ipos_df = remove_comp_suffix(tw_ipos_df)
tw_ipos_df

,symbol,company,exchange,proposed_share_price,shares_offered,priced_date,dollar_val_shares,deal_status,proposed_cap_classifiction,price_month_day,market_cap
0,ADAG,Adagene,NASDAQ Global,17.00-19.00,7354000,2021-02-09,1.606849e+08,expected,Micro-cap,Feb 09,160.7M
4,DSP,Viant Technology,NASDAQ Global,19.00-21.00,7500000,2021-02-10,1.811250e+08,expected,Micro-cap,Feb 10,181.1M
1,APR,Apria,NASDAQ Global Select,19.00-21.00,7500000,2021-02-11,1.811250e+08,expected,Micro-cap,Feb 11,181.1M
3,BMBL,Bumble,NASDAQ Global Select,28.00-30.00,34500000,2021-02-11,1.190250e+09,expected,Small-cap,Feb 11,1190.2M
6,SGFY,Signify Health,NYSE,17.00-19.00,23500000,2021-02-11,5.134750e+08,expected,Small-cap,Feb 11,513.5M


In [78]:
random_upcoming_ipo = [
    "check em out I guess",
    "I dunno who cares",
    "if you care",
    "are these good I dunno",
    "whats the point",
    "are any good?",
    "worth buying?",
    "are they overvalued",
    "any kewl ones?",
    "if you wanna dip a toe into em",
    "could be good or duds", 
    "this week",
    "... neat",
    "#stonks lmao right?",
    "from my robot",
    "coming at ya",
    "we'll see I guess", 
    "just cuz",
    "per the internet",
    "beep bop Im a robot",
    "schedule accordingly",
    "pass... or don't up to you",
    "at least thats the plan", 
    "have a good week",
    "kewl kewl kewl",
    "might buy all or none or some",
    "thanks for stopping by"
    ]

In [87]:
# get tweet text 
tweet_text = f"Upcoming IPOs {random.choice(random_upcoming_ipo)}\n"
for row in tw_ipos_df.itertuples():
    
    # remove .00 from proposed share prices to reduce unecesary characters
    psp = row.proposed_share_price.replace(".00", "")
    
    tweet_text += "\n"
    row_text = f"{row.price_month_day}: ${row.symbol} at ${psp}" 
    tweet_text += row_text
print(tweet_text)
print(f"******\nCharacter Count: {len(tweet_text)}")


Upcoming IPOs coming at ya

Feb 09: $ADAG at $17-19
Feb 10: $DSP at $19-21
Feb 11: $APR at $19-21
Feb 11: $BMBL at $28-30
Feb 11: $SGFY at $17-19
******
Character Count: 145


In [88]:
# table for tweet image
tw_ipos_img_df = tw_ipos_df[['price_month_day', 'symbol', 'company', 'proposed_share_price', 'market_cap']]
tw_ipos_img_df['proposed_share_price'] = "$" + tw_ipos_img_df['proposed_share_price']
tw_ipos_img_df = tw_ipos_img_df.rename(columns={'price_month_day': 'Date',
                                                'symbol': 'Symbol',
                                                'company': 'Company',
                                                'proposed_share_price': 'Proposed Price',
                                                'market_cap': 'Market Cap'
                                               })
tw_ipos_img_df

,Date,Symbol,Company,Proposed Price,Market Cap
0,Feb 09,ADAG,Adagene,$17.00-19.00,160.7M
4,Feb 10,DSP,Viant Technology,$19.00-21.00,181.1M
1,Feb 11,APR,Apria,$19.00-21.00,181.1M
3,Feb 11,BMBL,Bumble,$28.00-30.00,1190.2M
6,Feb 11,SGFY,Signify Health,$17.00-19.00,513.5M


In [92]:
# get tweet image
colorscale = [[0, '#118AB2'],[.5, '#FFFFFF'],[1, '#F2F2F2']]
fig =  ff.create_table(tw_ipos_img_df, colorscale=colorscale, height_constant=20)

fig.write_image("weekly_ipos.png", scale = 1)
fig.show()

In [93]:
# the name of the media file 
filename = "weekly_ipos.png"

# upload the file 
media = api.media_upload(filename) 

In [95]:
# Send out the tweet
api.update_with_media(filename="weekly_ipos.png", media_id=media.media_id, status=tweet_text)

Status(_api=<tweepy.api.API object at 0x00000200E93243C8>, _json={'created_at': 'Sun Feb 07 21:39:45 +0000 2021', 'id': 1358530915996225538, 'id_str': '1358530915996225538', 'text': 'Upcoming IPOs coming at ya\n\nFeb 09: $ADAG at $17-19\nFeb 10: $DSP at $19-21\nFeb 11: $APR at $19-21\nFeb 11: $BMBL at… https://t.co/mqqXCaswoU', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [{'text': 'ADAG', 'indices': [36, 41]}, {'text': 'DSP', 'indices': [60, 64]}, {'text': 'APR', 'indices': [83, 87]}, {'text': 'BMBL', 'indices': [106, 111]}], 'user_mentions': [], 'urls': [{'url': 'https://t.co/mqqXCaswoU', 'expanded_url': 'https://twitter.com/i/web/status/1358530915996225538', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [116, 139]}]}, 'source': '', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1285746488862269440, 'id_str': '1285746488862269440', 

# Stock Goes Live Today

In [ ]:
# get todays date
today = date(2021, 1, 21).strftime("%Y-%m-%d")  #date for testing, not actually today
# today = datetime.today().strftime("%Y-%m-%d")
print(today)

In [ ]:
# get stocks that go live today
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')
connection = engine.connect()
df = pd.read_sql(f"""
    SELECT s.*, 
        ci.city, 
        ci.state,
        ci.country,
        ci.website, 
        ci.industry,
        ci.sector, 
        ci.business_summary
    FROM stocks s
    LEFT JOIN company_info ci
        ON s.symbol = ci.symbol
    WHERE priced_date = '{today}'
        AND exchange <> 'NASDAQ Capital'""", connection)
connection.close()
df

In [ ]:
# industry portion of tweet
stock_industry = None
if df['industry'][0] is None:
    pass
elif df['industry'][0] == '':
    pass
else:
    stock_industry = df['industry'][0].lower()

    
print(stock_industry)

In [ ]:
# region portion of tweet
stock_region = None

# if no country information, pass
if df['country'][0] is None:
    pass
elif df['industry'][0] == '':
    pass
else:
    stock_industry = df['industry'][0].lower()

    
print(stock_region)

In [ ]:
tweet_text = f"{df['company'][0]} ${df['symbol'][0]} should trade today, a {stock_industry} company out of "

tweet_text

In [ ]:
# wrap text https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/autowrap.html

# Performance Analysis

In [ ]:
connection = engine.connect()
ipo_df = pd.read_sql("SELECT * FROM ipo_tracker.vw_stocks_performance", connection)
connection.close()
ipo_df.head()

## Sector and Industry

In [ ]:
# remove stocks with None or blank as sector / industry
si_data_df = ipo_df[(ipo_df["industry"].notnull()) & (ipo_df["industry"] != "")
            & (ipo_df["sector"].notnull()) & (ipo_df["sector"] != "")]
si_data_df.head(2)

### Sector by Year

In [ ]:
sector_by_year_df = si_data_df.pivot_table(index='sector', columns='first_trade_year', 
                                    aggfunc={"symbol": 'count'})
sector_by_year_df = sector_by_year_df.fillna(0).sort_values(by=sector_by_year_df.columns[-1], ascending=True)

# pass to function to flatten columns
sector_by_year_df = flatten_cols(sector_by_year_df)
sector_by_year_df


In [ ]:
ax = sector_by_year_df.plot(kind="barh", figsize=(8,7), color=colors, zorder=3, width=0.75)
ax.grid(which="major", axis='both', color='#e6e6e6', zorder=0)
ax.legend([2018, 2019, 2020])
ax.set_ylabel(f'Sector', fontsize=12)
ax.set_xlabel(f'IPOs Count', fontsize=12)
ax.set_title("IPO Count by Sector", fontsize=20)
plt.show()

## Top Performing n Period

Values for analysis

In [ ]:
# example stock to view
stock_symbol = 'ABCM' 

Data preparation

In [ ]:
# add price change column
sa_df = ipo_df.copy()
sa_df["first_close_to_rec_close"] = sa_df["most_recent_close"] / sa_df["first_day_close"] - 1
sa_df.head(5)

In [ ]:
# get sector for stock of interest
stock_sector = sa_df.loc[sa_df["symbol"] == stock_symbol]["sector"].iloc[0]
print(f"{stock_symbol} is in the {stock_sector} sector")

# get industry for stock of interest
stock_industry = sa_df.loc[sa_df["symbol"] == stock_symbol]["industry"].iloc[0]
print(f"{stock_symbol} is in the {stock_industry} industry")

# get market cap classification for stock of interest
stock_market_cap = sa_df.loc[sa_df["symbol"] == stock_symbol]["market_cap_classification"].iloc[0]
print(f"{stock_symbol} is a {stock_market_cap} market cap")

In [ ]:
# create new columns for vizualization
# sector
conditions = [(sa_df["symbol"] == stock_symbol), (sa_df["sector"] == stock_sector)]
choices = [stock_symbol, stock_sector]
sa_df["Stock Sector"] = np.select(conditions, choices, "Other")

# industry
conditions = [(sa_df["symbol"] == stock_symbol), (sa_df["industry"] == stock_industry)]
choices = [stock_symbol, stock_industry]
sa_df["Stock Industry"] = np.select(conditions, choices, "Other")

# market cap
conditions = [(sa_df["symbol"] == stock_symbol), (sa_df["industry"] == stock_market_cap)]
choices = [stock_symbol, stock_market_cap]
sa_df["Stock Market Cap"] = np.select(conditions, choices, "Other")

sa_df.head(2)

In [ ]:
# data for plotting sector in similar market cap
sa_mc_df = sa_df[sa_df["market_cap_classification"] == stock_market_cap]
sa_mc_df

In [ ]:
# Plot
colors = ["#118AB2", "#EF476F", "#d9d9d9"]
sns.lmplot('days_trading', 'first_close_to_rec_close', data=sa_mc_df, hue='Stock Sector', 
           palette=colors[0:3], scatter_kws={"s": 100}, fit_reg=True, height=6, aspect=1.6)
plt.title("Performance Over Time", fontsize=20)
plt.suptitle(f"{stock_market_cap} IPOs", fontsize=15)
plt.ylabel("YOY Change")
plt.xlabel("Days Trading")
plt.grid(which="major", axis='both', color='#e6e6e6', zorder=0)


In [ ]:
# remove stocks with None or blank as sector / industry
ax1 = sa_df.plot(kind="scatter", x="market_cap", y="first_close_to_rec_close", c=colors[0], figsize=(8,7), zorder=3)
ax.margins(0.05)

In [ ]:
sns.relplot(
    data=sa_df,
    x="days_trading", y="first_close_to_rec_close", col="market_cap_classification", col_wrap=3,
    hue="Stock Sector", palette=colors[0:3]
)

In [ ]:
# # table image with mpl option
# https://stackoverflow.com/questions/19726663/how-to-save-the-pandas-dataframe-series-data-as-a-figure

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# df = pd.DataFrame()
# df['date'] = ['2016-04-01', '2016-04-02', '2016-04-03']
# df['calories'] = [2200, 2100, 1500]
# df['sleep hours'] = [8, 7.5, 8.2]
# df['gym'] = [True, False, False]

# def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=12,
#                      header_color='#118AB2', row_colors=['#f1f1f2', 'w'], edge_color='w',
#                      bbox=[0, 0, 1, 1], header_columns=0,
#                      ax=None, **kwargs):
#     if ax is None:
#         size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
#         fig, ax = plt.subplots(figsize=size)
#         ax.axis('off')
#     mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
#     mpl_table.auto_set_font_size(False)
#     mpl_table.set_fontsize(font_size)

#     for k, cell in mpl_table._cells.items():
#         cell.set_edgecolor(edge_color)
#         if k[0] == 0 or k[1] < header_columns:
#             cell.set_text_props(weight='bold', color='w')
#             cell.set_facecolor(header_color)
#         else:
#             cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
#     return ax.get_figure(), ax

# fig,ax = render_mpl_table(tw_ipos_img_df, header_columns=0, col_width=2.5)
# fig.savefig("table_mpl.png")